Reading web-page and extracting table information

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content)

My_table = soup.find('table',{'class':'wikitable sortable'})

df = pd.DataFrame(columns=['Postcode', 'Borough','Neighbourhood'])
df_temp = []

i=0

#try clause to skip any companies with missing/empty board member tables
for row in My_table.find_all('tr'):
    for col in row.find_all('td'):
        df_temp.append(col.text.strip())
        
     
    if len(df_temp) > 0:
        df = df.append({'Postcode':df_temp[0], 'Borough':df_temp[1],'Neighbourhood':df_temp[2]}, ignore_index=True)
        df_temp = []


df.drop(df[df['Borough']=="Not assigned"].index, inplace = True)

Grouping inforation by Postcode and Borough

In [2]:
df.head()

df['Neighbourhood'] = df.groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
df.drop_duplicates(inplace = True)

Assigning Neighbourhood to missing ones 

In [3]:
df.loc[df['Neighbourhood']== 'Not assigned', 'Neighbourhood'] = df.loc[df['Neighbourhood']== 'Not assigned','Borough']

Result

df.head(10)

Adding latitue and longitude

In [4]:
import pandas as pd
import requests

url="data/Geospatial_Coordinates.csv"

c=pd.read_csv(url)
c.columns = ['Postcode', 'Latitude', 'Longitude']

df_ll = pd.merge (df , c, on = 'Postcode', how ='left' )

df_ll.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
